# Lesson 1 — Level 1 (Single cup)

Goal: build an **algorithm** (a sequence of moves) to drive around **one cup**.

You can use:
- `forward(seconds)`
- `backward(seconds)`
- `left(seconds)` / `right(seconds)`
- `turn_left(seconds)` / `turn_right(seconds)`
- `drift_left(seconds)` / `drift_right(seconds)`
- `horn()` (meep meep)

### Challenge
Write a plan that avoids the cup **without touching it**.

Tip: start slow. Then tighten it up.

In [ ]:
# Bootstrap (shared)
from pathlib import Path
import os, sys

def _safe_start() -> Path:
    try:
        return Path.cwd()
    except FileNotFoundError:
        return Path(os.environ.get('HOME', '/tmp'))

p = _safe_start().resolve()
for _ in range(30):
    cand = p / 'common' / 'lib'
    if cand.is_dir():
        if str(cand) not in sys.path:
            sys.path.insert(0, str(cand))
        break
    if p.parent == p:
        break
    p = p.parent

import bootstrap
if hasattr(bootstrap, 'init'):
    bootstrap.init(default_domain='0', verbose=True)
else:
    bootstrap.bootstrap(default_domain='0', verbose=True)


In [ ]:
import time
import robot_moves as rm

bot = rm.RobotMoves(base_speed=220, rate_hz=30)
bot.stop()
time.sleep(0.2)
print('Robot ready')

## Example: a tiny test plan
This is just to prove things move. You’ll write your own plan below.

In [ ]:
bot.horn()
bot.forward(0.5)
bot.stop()

## Your plan (edit this)

Make a list of steps. Each step is ("move_name", seconds).

Moves you can use as strings:
`forward`, `backward`, `left`, `right`, `turn_left`, `turn_right`, `drift_left`, `drift_right`

Then run the plan.

In [ ]:
# ============================
# YOUR PLAN (EDIT THIS)
# ============================

# Use simple commands like these:

# bot.horn()
# bot.forward(0.5)
# bot.backward(0.5)
# bot.left(0.5)
# bot.right(0.5)
# bot.turn_left(0.5)
# bot.turn_right(0.5)
# bot.drift_left(0.5)
# bot.drift_right(0.5)
# bot.stop()

# -----------------------------------
# Example plan (change this!)
# -----------------------------------

bot.horn()

bot.forward(0.7)
time.sleep(0.2)

bot.drift_left(0.8)
time.sleep(0.2)

bot.forward(0.6)
time.sleep(0.2)

bot.drift_right(0.8)
time.sleep(0.2)

bot.forward(0.5)

bot.stop()

print('Done')
